In [1]:
#Importing the sparksession which is mostly used for dataframes
from pyspark.sql import SparkSession

In [2]:
#creating  a spark session
spark= SparkSession.builder.appName('Spark Classification').getOrCreate()


#### Am going to use breast cancer datasets in sklearn for classification

In [3]:
#Downloading the dataset
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()

In [4]:
#checking the content of the dataset
cancer.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [5]:
#Creating the dataframe using pandas
import pandas as pd
data=pd.DataFrame(cancer['data'],columns=cancer['feature_names'])

#### Brief description of the dataset we are using

In [6]:
print(cancer['DESCR'])

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry 
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 3 is Mean Radius, f

In [7]:
#Gettting the target columns from the brest cancer dataset
target=cancer['target'] 

In [8]:
#appending the target column to the dataframe
data['target']=target

In [9]:
# The values we are trying to predict 
print(cancer['target_names'])

['malignant' 'benign']


In [10]:
data.head(5)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [11]:
data.shape

(569, 31)

In [12]:
#coverting to spark dataframes
data=spark.createDataFrame(data)

In [13]:
data.show(5)

+-----------+------------+--------------+---------+---------------+----------------+--------------+-------------------+-------------+----------------------+------------+-------------+---------------+----------+----------------+-----------------+---------------+--------------------+--------------+-----------------------+------------+-------------+---------------+----------+----------------+-----------------+---------------+--------------------+--------------+-----------------------+------+
|mean radius|mean texture|mean perimeter|mean area|mean smoothness|mean compactness|mean concavity|mean concave points|mean symmetry|mean fractal dimension|radius error|texture error|perimeter error|area error|smoothness error|compactness error|concavity error|concave points error|symmetry error|fractal dimension error|worst radius|worst texture|worst perimeter|worst area|worst smoothness|worst compactness|worst concavity|worst concave points|worst symmetry|worst fractal dimension|target|
+-----------

In [14]:
data.printSchema()

root
 |-- mean radius: double (nullable = true)
 |-- mean texture: double (nullable = true)
 |-- mean perimeter: double (nullable = true)
 |-- mean area: double (nullable = true)
 |-- mean smoothness: double (nullable = true)
 |-- mean compactness: double (nullable = true)
 |-- mean concavity: double (nullable = true)
 |-- mean concave points: double (nullable = true)
 |-- mean symmetry: double (nullable = true)
 |-- mean fractal dimension: double (nullable = true)
 |-- radius error: double (nullable = true)
 |-- texture error: double (nullable = true)
 |-- perimeter error: double (nullable = true)
 |-- area error: double (nullable = true)
 |-- smoothness error: double (nullable = true)
 |-- compactness error: double (nullable = true)
 |-- concavity error: double (nullable = true)
 |-- concave points error: double (nullable = true)
 |-- symmetry error: double (nullable = true)
 |-- fractal dimension error: double (nullable = true)
 |-- worst radius: double (nullable = true)
 |-- worst 

In [15]:
# Describing the data
data.toPandas().describe().transpose()

,count,mean,std,min,25%,50%,75%,max
mean radius,569.0,14.127292,3.524049,6.981000,11.700000,13.370000,15.780000,28.11000
mean texture,569.0,19.289649,4.301036,9.710000,16.170000,18.840000,21.800000,39.28000
mean perimeter,569.0,91.969033,24.298981,43.790000,75.170000,86.240000,104.100000,188.50000
mean area,569.0,654.889104,351.914129,143.500000,420.300000,551.100000,782.700000,2501.00000
mean smoothness,569.0,0.096360,0.014064,0.052630,0.086370,0.095870,0.105300,0.16340
mean compactness,569.0,0.104341,0.052813,0.019380,0.064920,0.092630,0.130400,0.34540
mean concavity,569.0,0.088799,0.079720,0.000000,0.029560,0.061540,0.130700,0.42680
mean concave points,569.0,0.048919,0.038803,0.000000,0.020310,0.033500,0.074000,0.20120
mean symmetry,569.0,0.181162,0.027414,0.106000,0.161900,0.179200,0.195700,0.30400
mean fractal dimension,569.0,0.062798,0.007060,0.049960,0.057700,0.061540,0.066120,0.09744


In [16]:
columns=data.columns[:-1]
columns

['mean radius',
 'mean texture',
 'mean perimeter',
 'mean area',
 'mean smoothness',
 'mean compactness',
 'mean concavity',
 'mean concave points',
 'mean symmetry',
 'mean fractal dimension',
 'radius error',
 'texture error',
 'perimeter error',
 'area error',
 'smoothness error',
 'compactness error',
 'concavity error',
 'concave points error',
 'symmetry error',
 'fractal dimension error',
 'worst radius',
 'worst texture',
 'worst perimeter',
 'worst area',
 'worst smoothness',
 'worst compactness',
 'worst concavity',
 'worst concave points',
 'worst symmetry',
 'worst fractal dimension']

In [17]:
#Importing libraries for asemblering and categorizing label column
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler

In [18]:
#creating a vector matrix for feeding in the model
vector = VectorAssembler(inputCols=columns,outputCol='features')
vdata=vector.transform(data)
vdata.select('features').show(5)

+--------------------+
|            features|
+--------------------+
|[17.99,10.38,122....|
|[20.57,17.77,132....|
|[19.69,21.25,130....|
|[11.42,20.38,77.5...|
|[20.29,14.34,135....|
+--------------------+
only showing top 5 rows



In [19]:
#Finding the number of unique values being predicted in the target column
vdata.select('target').distinct().show()

+------+
|target|
+------+
|     0|
|     1|
+------+



In [20]:
#categorizing/indexing  the target column
indexer=StringIndexer(inputCol="target",outputCol="label")
ssdata=indexer.fit(vdata)
ssdata=ssdata.transform(vdata)
ssdata.select('target','label').show(5)


+------+-----+
|target|label|
+------+-----+
|     0|  1.0|
|     0|  1.0|
|     0|  1.0|
|     0|  1.0|
|     0|  1.0|
+------+-----+
only showing top 5 rows



In [21]:
# #Finding correlation matrix between the Columns in the dataframe
# features=data.rdd.map(lambda row: row[0:])
# from pyspark.mllib.stat import Statistics
# corr_mat=Statistics.corr(features, method='pearson')

In [22]:
#print(corr_mat)

 ##                     Creating the Models

#### Naive Bayes

In [23]:
# importing naivebayes model
from pyspark.ml.classification import NaiveBayes
#importing the libarry for evaluating the model
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [24]:
#splitting the dataset
splits=ssdata.randomSplit([0.7,0.3],1)
train=splits[0]
test=splits[1]

In [25]:
vdata.count(), train.count() , test.count()

(569, 400, 169)

In [26]:
#creating the model
naive=NaiveBayes(featuresCol='features',labelCol='label',modelType='multinomial')
model=naive.fit(train)

In [27]:
pred=model.transform(test)
pred.select('label','prediction').show(20)

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  1.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  1.0|       0.0|
|  1.0|       0.0|
|  1.0|       1.0|
+-----+----------+
only showing top 20 rows



In [28]:
#Evaluating the model
evaluator=MulticlassClassificationEvaluator(labelCol='label',predictionCol='prediction',metricName='accuracy')
accuracy=evaluator.evaluate(pred)
accuracy

0.8994082840236687

## Decision Tree Classifier

In [29]:
# importin decision tree classifier
from pyspark.ml.classification import DecisionTreeClassifier

In [30]:
#creating the model
dt=DecisionTreeClassifier(featuresCol='features',labelCol="label")
dtmodel=dt.fit(train)

In [31]:
#predicting the values
predictions=dtmodel.transform(test)
predictions.select('label','prediction').show(10)

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
+-----+----------+
only showing top 10 rows



In [32]:
#Evaluating the model
evaluator=MulticlassClassificationEvaluator(labelCol='label',predictionCol='prediction',metricName='accuracy')
dtaccuracy=evaluator.evaluate(predictions)
dtaccuracy

0.9704142011834319

#### From the above clasification methods, decision tree proves to be the best classification method since it has the highest accuracy    